# Plot Chebyshev Coeffs

- Compare the plots of value
    - note: if you modify line 74 and add a small number, you can see the green line (exact) perturbed. In general, it traces (-16, 16, 32) exactly
    
- Compare the plots of error

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List
plt.rcParams["figure.figsize"] = (20,15)
def sigmoid(z):
    return 1/(1 + np.exp(-z))

class Config():
    def __init__(self, name, start, end, degree):
        self.name = name
        self.start = start
        self.end = end
        self.degree = degree
        self.data = None

        
    def release(self, bound_start, bound_end):
        if not (self.start <= bound_start <= bound_end <= self.end):
            raise Exception()
        return f"raw_data/sigmoidResults_{self.end}_{self.degree}.txt"
    
    def filter_data(self, start, end):
        idxs = self.data[:, 0]
        mask = (start < idxs) & (idxs < end)
        idxs_filtered = self.data[mask][:, 0]
        return self.data[mask][:, 1], idxs_filtered
        
conf_list = [
    # Config("Degree16", -16, 16, 32)
    Config("Degree128", -64, 64, 128),
    Config("Degree119", -64, 64, 119)
]

# Value plot

In [ ]:
def plot_range(config_list: List[Config], start, end):
    filtered_data_list = []
    for config in config_list:
        config.data = np.loadtxt(
            config.release(start, end),
            delimiter=",",
            dtype=np.float64
        )
        
        filtered_data, idxs_filtered = config.filter_data(start, end)
        
        plt.plot(idxs_filtered, filtered_data, label=config.name)
        
    # Now plot the exact value
    exact = sigmoid(idxs_filtered)
    plt.plot(idxs_filtered, exact, label="exact")

    # Naming the x-axis, y-axis and the whole graph
    plt.xlabel("X-value")
    plt.ylabel("Sigmoid(x)")
    plt.title(f"Cheb Approx on specified range ({start}, {end})")
    plt.grid()

    # Adding legend, which helps us recognize the curve according to it's color
    plt.legend()

    plt.savefig(f"approx_plots/compare_across_range_{start}_{end}.png")
    # To load the display window
    plt.show()

In [ ]:
for range_end in [16]:
    plot_range(config_list=conf_list, start=-range_end, end=range_end,)

# Error plot

In [ ]:
def plot_error_compare(config_list, start, end):
    
    filtered_data_list = [] 
    for config in config_list:
        config.data = np.loadtxt(
            config.release(start, end),
            delimiter=",",
            dtype=np.float64
        )
        
        filtered_data, idxs_filtered = config.filter_data(start, end)
        filtered_data_list.append(filtered_data)
        
        exact_filtered = sigmoid(idxs_filtered)
        
        
        label = f"Exact - Approx(start={config.start}, end={config.end}, degree={config.degree}) across range ({start}, {end})"
        diff = exact_filtered - filtered_data
        
        plt.plot(idxs_filtered, diff, label=label)
        
        max_diff_idx = np.argmax(diff)
        max_diff = diff[max_diff_idx]
        idx = idxs_filtered[max_diff_idx]
        plt.scatter(idx, max_diff, s=60)
        print(max_diff)
        plt.annotate(f"{max_diff:.6f}", (idx, max_diff + 0.000015))


    # Naming the x-axis, y-axis and the whole graph
    plt.xlabel("X-value")
    plt.ylabel("Error")
    plt.title(f"Exact-Approx Plot")
    # plt.grid()

    # Adding legend, which helps us recognize the curve according to it's color
    plt.legend(loc='upper left')

    plt.savefig(f"approx_plots/error_on_range_{start}_{end}.png", bbox_inches="tight")
    # To load the display window
    plt.show()
    
if False:
    
    conf_list = [
        Config("Degree128", -64, 64, 128),
        Config("Degree119", -64, 64, 119)
    ]
    plot_error_compare(conf_list, -64, 64)
else:
    
    conf_list = [
        Config("Degree128", -16, 16, 32),
        Config("Degree119", -16, 16, 59)
    ]
    plot_error_compare(conf_list, -16, 16)

In [ ]:
def plot_stacked(config_list: List[Config], start, end):
    fig, axs = plt.subplots(2, sharex=True)
    ax1, ax2 = axs
    
    ax1.grid()
    ax1.set(xlabel="x", ylabel="sigmoid(x)")
    ax1.set_title(f"Plot of Cheb_Approx on specified range ({start}, {end})")

    
    ax2.grid()
    ax2.set(xlabel="x", ylabel="Error")
    
    ax2.set_title(f"Plot of Difference between Exact and Cheb_Approx on specified range ({start}, {end})")
        
    filtered_data_list = []
    for config in config_list:
        config.data = np.loadtxt(
            config.release(start, end),
            delimiter=",",
            dtype=np.float64
        )
        
        filtered_data, idxs_filtered = config.filter_data(start, end)
        filtered_data_list.append(filtered_data)
        
        
        ax1.plot(idxs_filtered, filtered_data, label=config.name)
        
        # Yes, this is inefficient but it'll do for now
        exact_filtered = sigmoid(idxs_filtered)
        ax2.plot(
            idxs_filtered, exact_filtered - filtered_data, 
            label=f"Exact-Approx(start={config.start}, end={config.end}, degree={config.degree}))"
        )
        
        
    
    # Now plot the exact value
    exact = sigmoid(idxs_filtered)
    ax1.plot(idxs_filtered, exact, label="Exact")    
    
    ax1.legend()
    ax2.legend()
    plt.savefig(f"approx_plots/stacked_plot_approx_over_error_on_range_{start}_{end}.png")
    # To load the display window
    plt.show()

In [ ]:
config_list

for range_end in [64, 32, 16, 8, 4, 2, 1, 0.5]:
    plot_stacked(config_list=conf_list, start=-range_end, end=range_end,)